### Please apply following rules: (Indicator - BUY, CALL, LONG)
    * Make sure the red line of MACD is above - 0
    * Make a bollinger band of simple moveing average - with 10 days MA and 1 as STD DEV
    * (T - 1) Yesterday's High of the stock is below the lower bound of Bollinger's Band
    * T open should not be above T -1 high
    * The stock's in T should cross T - 1's high, then buy it at T-1's high and put a stop loss of T-1's low
    * If the stock doesnt cross T-1's high in T, then dont buy it.
    * when the next price drop happens (Red candle day), but is above the stop loss; revise the stop loss 
        to the low of that day
    

### Calculation of Quantity and PnL

number of shares to buy: lpt(loss per trade)/(600-585)

ltp = 1000 (By Appetite)
    
number of share to buy = 1000/15 

buy price - initial stop loss = (600 - 585)  = (high of T-1  - low of T-1)

buy price = 600

sell price = 617

profit per share = 17

profit % = 17/600 = 2.8%

investment = 65 * 600 ~ 3900

net profit ~ 750 

In [1]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import sys
# sys.path.append('./python_code')
# import featexp as fe
from sklearn.model_selection import train_test_split
from pandas_datareader import data as web 
import requests as req 
import pandas as pd
import StringIO as strio
import datetime as dt
from tqdm import tqdm_notebook as tqdm
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column

Loading BokehJS ...

In [2]:
url = "https://www.nseindia.com/content/indices/ind_nifty50list.csv"
csvfromurl = req.get(url).content
csvfordf = pd.read_csv(strio.StringIO(csvfromurl.decode('utf-8')))
nifty50fd = pd.DataFrame(csvfordf)
nifty50fd

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finance Ltd.,FINANCIAL SERVICES,BAJFINANCE,EQ,INE296A01024
5,Bajaj Finserv Ltd.,FINANCIAL SERVICES,BAJAJFINSV,EQ,INE918I01018
6,Bharat Petroleum Corporation Ltd.,ENERGY,BPCL,EQ,INE029A01011
7,Bharti Airtel Ltd.,TELECOM,BHARTIARTL,EQ,INE397D01024
8,Bharti Infratel Ltd.,TELECOM,INFRATEL,EQ,INE121J01017
9,Britannia Industries Ltd.,CONSUMER GOODS,BRITANNIA,EQ,INE216A01030


In [7]:
backtestdt = dt.date.today()
startdate = dt.date(2019, 1, 1)
enddate = dt.date(2019, 4, 14)

In [3]:
%%time

stock_df = pd.DataFrame()

diffyears = 0.0
diffyears = enddate.year - startdate.year
for eachscrip in tqdm(nifty50fd['Symbol']):
    stock = eachscrip+'.NS'
    try:
#         print "Processing:", stock
        stockdata = web.DataReader(stock, data_source='yahoo',start=startdate, end=enddate)
        stockdata['Symbol'] = eachscrip
        stock_df = stock_df.append(stockdata)
    except:
        print "Error Processing:", stock
            
print stock_df.shape


(2998, 7)
CPU times: user 6.95 s, sys: 342 ms, total: 7.29 s
Wall time: 1min 28s


In [4]:
def ExpMovingAverage(values, window):
    """ Numpy implementation of EMA
    """
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    a =  np.convolve(values, weights, mode='full')[:len(values)]
    a[:window] = a[window]
    return a

In [5]:
# MACD 12, 26, 9
def rolling_mean(df_input):
    df2 = df_input.Close.rolling(window=10).agg({"cls_mean": np.mean, "cls_std": np.std})
    df2 = df2.fillna(method='bfill')
    df2 = pd.concat([df_input, df2], axis=1)
    df2['lower'] = df2.cls_mean - df2.cls_std
    df2['upper'] = df2.cls_mean + df2.cls_std
    df2['26ema'] = ExpMovingAverage(df_input.Close, 26)
    df2['12ema'] = ExpMovingAverage(df_input.Close, 12)
    df2['MACD'] = (df2['12ema'] - df2['26ema'])
    df2['Signal'] = ExpMovingAverage(df2['MACD'], 9)
    
    return df2

In [8]:
for eachscrip in tqdm(nifty50fd['Symbol']):
    stock = eachscrip
    try:
        stock_df[stock_df.Symbol == stock].Close.rolling(window=10).agg({"cls_mean": np.mean, "cls_std": np.std})
        ExpMovingAverage(stock_df[stock_df.Symbol == stock].Close, 12)
    except:
        print "error processing", stock
        stock_df = stock_df[stock_df.Symbol != stock]
        
df = stock_df

df = df.reset_index()

df = df.groupby('Symbol').apply(rolling_mean)
df_transformed = df.reset_index()

# Look at End Date
df_transformed = df_transformed[df_transformed.Date== enddate] 

# Check MACD Signal > 0 For Buy oppertunities
df_transformed = df_transformed[df_transformed.Signal > 0]

# Check if the Days High is below lower bound of MACD
final_df = df_transformed[df_transformed.High < df_transformed.lower]
final_df

,index,Date,High,Low,Open,Close,Volume,Adj Close,Symbol,cls_mean,cls_std,lower,upper,26ema,12ema,MACD,Signal


In [ ]:
source = ColumnDataSource(df.reset_index())

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p = figure(tools=TOOLS)

p.scatter(x='Date', y='Close', line_color=None, fill_alpha=0.3, size=5, source=source)

# band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
#             fill_alpha=1.0, line_width=1, line_color='black')
# p.add_layout(band)

p.title.text = "Rolling Standard Deviation"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'X'
p.yaxis.axis_label = 'Y'

show(p)